# DataPipeline 
This notebook is used for fetching and cleaning the price data which will be used in the backtest of the model

### Dimensions of data:
1) 15 minute candles
2) 100 most populair trading pairs
3) 18 months of data

### 0) Parameters for Data 

In [1]:
MONTHS_OF_DATA = 18
AMOUNT_OF_PRODUCTS = 100

### 1) Preparations for data download

#### 1.1) Imports and API keys

In [4]:
from binance import Client
import pandas as pd
import datetime as dt
import numpy as np
import json
from os.path import exists
from tqdm import trange, tqdm

In [5]:
with open('API_keys.json') as f:
    data = json.load(f)
api_key = data['API-Key']
api_secret = data['Secret-Key']

#### 1.2) Connecting to the binance API

In [23]:
client = Client(api_key, api_secret)
candle = client.KLINE_INTERVAL_15MINUTE
futures = True

### 2) Data download and cleaning

#### 2.1) Create list of products to add

In [27]:
# first fetch all products then filter them for: USDT market then for 
if not futures:
    dictionary              = client.get_products()['data']
    df_products             = pd.DataFrame(dictionary)
    df_filtered_products    = df_products[(df_products['q'] == 'USDT')&(df_products['st'] == 'TRADING')]
    df_sorted_products      = df_filtered_products.sort_values(by='qv', ascending = False)
    #save the list of products to use in a list
    list_of_tickers         = list(df_sorted_products.s.head(AMOUNT_OF_PRODUCTS).values)
else:
    dictionary              = client.futures_exchange_info()
    df_products             = pd.DataFrame(dictionary['symbols'])
    df_filtered_products    = df_products[(df_products['contractType'] == 'PERPETUAL')&(df_products['status'] == 'TRADING')]
    list_of_tickers         = list(df_filtered_products.symbol.values)

#### 2.2) Data downloader

,0,1,2,3,4,5,6,7,8,9,10,11
0,1567964700000,10000,10000,10000,10000,0.002,1567965599999,20,2,0.001,10,0
1,1567965600000,10000,10000,10000,10000,0,1567966499999,0,0,0,0,0
2,1567966500000,10000,10000,10000,10000,0,1567967399999,0,0,0,0,0
3,1567967400000,10000,10000,10000,10000,0,1567968299999,0,0,0,0,0
4,1567968300000,10000,10000,10000,10000,0,1567969199999,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
127546,1682756100000,29307.70,29323.60,29250.00,29285.40,3338.196,1682756999999,97751709.09410,31680,1357.102,39745030.33880,0
127547,1682757000000,29285.40,29308.70,29266.00,29306.70,2260.687,1682757899999,66206255.97710,22700,1276.427,37385886.21070,0
127548,1682757900000,29306.80,29322.60,29290.50,29300.00,1590.837,1682758799999,46625535.59830,18621,926.268,27148162.08010,0
127549,1682758800000,29300.00,29310.00,29240.30,29261.10,4234.161,1682759699999,123939199.59100,34176,1749.453,51214570.33140,0


In [37]:
import datetime as dt
#datetime.datetime()
dt.datetime.fromtimestamp(1567964700000/1000.0)

datetime.datetime(2019, 9, 8, 19, 45)

In [ ]:
end = dt.datetime.today()- dt.timedelta(days=1)
start = end - dt.timedelta(days=30*MONTHS_OF_DATA)

END = end.strftime('%-d %b, %Y')
START = start.strftime('%-d %b, %Y')

In [38]:
rename_map = {
        0: 'open_time',
        1: 'open', 
        2: 'high', 
        3: 'low', 
        4: 'close', 
        5: 'volume',
        6: 'close_time', 
        7: 'qav', 
        8: 'num_trades',
        9: 'taker_base_vol', 
        10: 'taker_quote_vol', 
        11: 'ignore'
    }
# purpose of function is to fetch price data for a specific symbol
def get_klines_data(symbol):
    klines = client.get_historical_klines(symbol, candle, START, END)
    data = pd.DataFrame(klines).rename(rename_map, axis= 1) 
    data.open_time = [dt.datetime.fromtimestamp(x/1000.0) for x in data.open_time]
    data.close_time = [dt.datetime.fromtimestamp(x/1000.0) for x in data.close_time]
    return data

def get_futures_data(symbol):
    klines = client.futures_historical_klines(symbol, candle, '4 year ago UTC')
    data = pd.DataFrame(klines).rename(rename_map, axis= 1) 
    data.open_time = [dt.datetime.fromtimestamp(x/1000.0) for x in data.open_time]
    data.close_time = [dt.datetime.fromtimestamp(x/1000.0) for x in data.close_time]
    return data

In [40]:
#searches for the historic data for all the specified tickers 
for ticker in tqdm(list_of_tickers):
    if not futures:
        path = 'price_data_spot/{}.csv'.format(ticker)
        #checks if it still needs to save data
        if not exists(path):
            #fetch data
            symbol_data = get_klines_data(ticker)
            #save data
            symbol_data.to_csv(path)
    else:
        path = 'price_data_futures/{}.csv'.format(ticker)
        #checks if it still needs to save data
        if not exists(path):
            #fetch data
            symbol_data = get_futures_data(ticker)
            #save data
            symbol_data.to_csv(path)

100%|██████████| 200/200 [4:29:44<00:00, 80.92s/it]  


In [19]:
import os
list_of_tickers = os.listdir('price_data')

#### 2.3) Merge all data

In [42]:
price_data = False
for idx, ticker in tqdm(enumerate(list_of_tickers)):
    ticker = ticker.split('.')[0]
    if not futures:
        path = 'price_data_spot/{}.csv'.format(ticker)
    else:
        path = 'price_data_futures/{}.csv'.format(ticker)
    data = pd.read_csv(path)
    #if this is first loop instantiate the dataframe
    if idx == 0:
        price_data = data[['close_time','close']]
        price_data = price_data.set_index('close_time')
        price_data = price_data.rename({'close':ticker},axis=1)
    else:
        data = data[['close_time','close']]
        data = data.set_index('close_time')
        data = data.rename({'close':ticker},axis=1)
        data = data[~data.index.duplicated(keep='first')]
        #now merge the dataframes
        #price_data = pd.merge(left=price_data,right=data,how='left',on='close_time')
        price_data = price_data.join(data, how='outer')

    price_data = price_data[~price_data.index.duplicated(keep='first')]

200it [01:51,  1.79it/s]


#### 2.4) Clean dataset

In [49]:
#cleaning of the dataset
#price_data = price_data.dropna(thresh=100, axis=1,inplace=False)
#price_data = price_data.dropna(axis=1, thresh=len(price_data)-2)
#price_data = price_data.dropna()
#rounding the index
price_data.index = pd.to_datetime(price_data.index).round('min')
#drop the stablecoins
not_allowed = ['BUSDUSDT','TUSDUSDT','USDCUSDT','PAXUSDT','USDCTUSD','USDCPAX','PAXTUSD','BTCNGN', 'EURUSDT','GBPUSDT', 'AUDUSDT']
price_data = price_data.drop(not_allowed, axis=1, errors = 'ignore')

#### 2.5) Calculate the returns and log-returns for the dataset

In [50]:
price_data = price_data[(price_data.index.hour % 12 == 0)&(price_data.index.minute  == 0)]

In [51]:
price_data

,BTCUSDT,ETHUSDT,BCHUSDT,XRPUSDT,EOSUSDT,LTCUSDT,TRXUSDT,ETCUSDT,LINKUSDT,XLMUSDT,...,ARBUSDT,JOEUSDT,TLMUSDT,AMBUSDT,LEVERUSDT,RDNTUSDT,HFTUSDT,XVSUSDT,ETHBTC,BLURUSDT
close_time,,,,,,,,,,,,,,,,,,,,,
2019-09-09 00:00:00,10391.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-09 12:00:00,10248.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-10 00:00:00,10289.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-10 12:00:00,10248.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-11 00:00:00,10108.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-27 12:00:00,28920.10,1880.60,117.35,0.4579,1.017,88.09,0.06533,19.521,7.027,0.09297,...,1.3958,0.5016,0.01938,0.01292,0.001843,0.4089,0.6020,6.070,0.06518,NaN
2023-04-28 00:00:00,29631.00,1914.23,117.19,0.4657,1.030,89.89,0.06571,19.833,7.160,0.09441,...,1.4313,0.5114,0.01988,0.01346,0.001926,0.4157,0.6149,6.324,0.06472,NaN
2023-04-28 12:00:00,29187.30,1905.79,117.03,0.4710,1.029,89.25,0.06606,19.690,7.107,0.09483,...,1.4165,0.5033,0.01956,0.01325,0.001895,0.4024,0.6017,6.154,0.06534,NaN


In [24]:
log_ret = np.log(price_data / price_data.shift(1))
#log_ret.dropna(inplace=True)

In [25]:
pct_ret = price_data.pct_change(1)
#pct_ret.dropna(inplace=True)

#### 2.6) Save the data

In [26]:
#log_ret.to_csv('log_returns.csv')
#pct_ret.to_csv('pct_returns.csv')
price_data.to_csv('futures_data.csv')